In [1]:
from IPython import get_ipython


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
import json as js



In [3]:
import random
import math
from random import randint,uniform



In [4]:
class PointsCollection:
    def __init__(self, points = [], color = None, marker = None, size = 3):
        self.points = np.array(points)
        self.color = color
        self.marker = marker
        self.s = size
        
class LinesCollection:
    def __init__(self, lines = [], color = None):
        self.color = color
        self.lines = lines
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        if self.color:
            return mcoll.LineCollection(self.lines, [mcolors.to_rgba(self.color)] * len(self.lines))
        else:
            return mcoll.LineCollection(self.lines)

class Plot:
    def __init__(self, points=[], lines=[], json = None):
        if json is None:
            self.points = points
            self.lines = lines
        else:
            self.points = [PointsCollection(pointsCol) for pointsCol in js.loads(json)["points"]]
            self.lines = [LinesCollection(linesCol) for linesCol in js.loads(json)["lines"]]
            
    def draw(self, title = None):
        ax = plt.axes()
        for collection in self.points:
            if collection.points.size > 0:
                ax.scatter(*zip(*collection.points), c=collection.color, marker=collection.marker, s = collection.s)
        for collection in self.lines:
            ax.add_collection(collection.get_collection())
        ax.autoscale()

        if title is not None:
            plt.title(label= title)
        
        plt.draw()
        
    def toJSON(self):
        return js.dumps({"points": [pointCol.points.tolist() for pointCol in self.points], 
                          "lines":[linesCol.lines for linesCol in self.lines]})



In [5]:


# zbiór a
def aDataset():
    cords = (-1000, 1000)
    d = [(uniform(cords[0], cords[1]), uniform(cords[0], cords[1])) for _ in range(10**5)]
    
    # Plot([PointsCollection(a)]).draw()
    with open("a.txt", "w") as fp:
        js.dump(d, fp)

    return d



In [6]:



# zbiór b
def bDataset():
    cords = (-10 ** 14, 10 ** 14)

    d = [(uniform(cords[0], cords[1]), uniform(cords[0], cords[1])) for _ in range(10**5)]
    # Plot([PointsCollection(d)]).draw()

    with open("b.txt", "w") as fp:
        js.dump(d, fp)
    
    return d



In [7]:


# zbior c okrag
def cDataset():
    n = 1000
    r = 100
    d = [None] * n
    p = (0.0, 0.0)
    for i in range(n):
        alpha = random.random() * (2*math.pi)
        d[i] = (p[0] + r*math.sin(alpha), p[1] + r*math.cos(alpha))

    # Plot([PointsCollection(d)]).draw()

    with open("c.txt", "w") as fp:
        js.dump(d, fp)
    
    return d



In [8]:



# zbior d prosta
def dDataset():
    n = 1000

    p1 = (-1.0, 0.0)
    p2 = (1.0, 0.1)

    lineA = (p1[1] - p2[1]) / (p1[0] - p2[0])
    lineB = p1[1] - lineA * p1[0]

    d = [None] * n
    for i in range(n):
        x = uniform(-1000,1000)
        d[i] = (x, lineA*x + lineB)

    # Plot([PointsCollection(d)]).draw()

    with open("d.txt", "w") as fp:
        js.dump(d, fp)
    
    return d



In [9]:

def plotDataset(d):



    Plot([PointsCollection(d)]).draw()



In [10]:
def det3x3(a,b,c):
    return a[0]*b[1] + a[1]*c[0] + b[0]*c[1] - c[0]*b[1] - a[1]*b[0] - a[0]*c[1]



In [11]:
def det2x2(a,b,c):
    return (a[0] - c[0])*(b[1] - c[1]) - (a[1] - c[1])*(b[0] - c[0])



In [12]:
def detNp3x3(a,b,c):
    arr = np.array([[a[0], a[1], 1], [b[0], b[1], 1], [c[0], c[1], 1]])
    return np.linalg.det(arr)



In [13]:
def detNp2x2(a,b,c):
    arr = np.array([[a[0] - c[0], a[1] - c[1]], [b[0] - c[0], b[1] - c[1]]])
    return np.linalg.det(arr)



In [14]:
def classify(points, detFun, e = 10** -14, a = (-1.0,0.0), b = (1.0,0.1)):
    left = []
    right = []
    collinear = []

    for p in points:
        d = detFun(a,b,p)

        if d > e:
            left.append(p)
        elif d < -e:
            right.append(p)
        else:
            collinear.append(p)
 
    return left, collinear, right



In [15]:
def plotClassification(points, detFun, e = 10**-14, a = (-1.0,0.0), b = (1.0,0.1)):
    
    left, collinear, right = classify(points, detFun, e, a, b)

    
    classified = [PointsCollection(left, color='blue'),
                  PointsCollection(right, color='green'),
                  PointsCollection(collinear, color= 'pink')]



    Plot(classified).draw()

    print("Left: ", len(left))
    print("Right: ",len(right))
    print("Collinear: ",len(collinear))



In [16]:
def countDiff(points, detFun1, detFun2, e = 10**-14, a = (-1.0,0.0), b = (1.0,0.1), show = True):
    #czerwone to punkty sklasyfikowane przez detFun1 jako wspolliniowe, a detFun2 nie
    #zielone to punkyu sklasyfikowane przez detFun2 jako wpolliniowe, a detFun1 nie
    def cat(p, f):
        d = f(a,b,p)

        if d > e:
            return 'l'
        elif d < -e:
            return 'r'
        else:
            return 'c'
    res1 = []
    res2 = []        
    for p in points:
        if cat(p,detFun1) is not cat(p,detFun2):
            if cat(p, detFun1) is not 'c':
                res2.append(p)
            else:
                res1.append(p)
            

    if show is True:
        get_ipython().run_line_magic('matplotlib', 'ipympl')
        Plot([PointsCollection(res1, color= 'red'),PointsCollection(res2, color='green')]).draw()

    print ("Difference:",len(res1)+len(res2))



In [17]:
from pathlib import Path
def load():
    file = Path("a.txt")
    if file.is_file():
        with open(file, "r") as fp:
            a = js.load(fp)
    else:
        a = aDataset()
    
    file = Path("b.txt")
    if file.is_file():
        with open(file, "r") as fp:
            b = js.load(fp)
    else:
        b = bDataset()
    
    file = Path("c.txt")
    if file.is_file():
        with open(file, "r") as fp:
            c = js.load(fp)
    else:
        c = cDataset()
    
    file = Path("d.txt")
    if file.is_file():
        with open(file, "r") as fp:
            d = js.load(fp)
    else:
        d = dDataset()

    return a, b, c, d
a,b,c,d = load()



In [18]:
def resetDatasets():
    global a,b,c,d
    a = aDataset()
    b = bDataset()
    c = cDataset()
    d = dDataset()



In [19]:
%matplotlib ipympl

plotDataset(a)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
%matplotlib widget

plotDataset(b)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
%matplotlib widget

plotDataset(c)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
%matplotlib widget

plotDataset(d)













Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
epsilon = 0



In [24]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [25]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [26]:
%matplotlib widget

countDiff(a,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [27]:
%matplotlib widget
countDiff(a,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [28]:
%matplotlib widget
countDiff(a,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [29]:
epsilon = 10**-14



In [30]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [31]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [32]:
%matplotlib widget
countDiff(a,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [33]:
%matplotlib widget
countDiff(a,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [34]:
%matplotlib widget
countDiff(a,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [35]:
epsilon = 10**-5



In [36]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [37]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [38]:
%matplotlib widget
countDiff(a,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [39]:
%matplotlib widget
countDiff(a,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [40]:
%matplotlib widget
countDiff(a,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [41]:
epsilon = 10**-2



In [42]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [43]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
%matplotlib widget

plotClassification(a,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49961
Right:  50039
Collinear:  0


In [44]:
%matplotlib widget
countDiff(a,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [45]:
%matplotlib widget
countDiff(a,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [46]:
%matplotlib widget
countDiff(a,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [47]:
epsilon = 1



In [48]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(a,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49935
Right:  50015
Collinear:  50


In [49]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
plotClassification(a,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49935
Right:  50015
Collinear:  50


In [50]:
countDiff(a,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [51]:
countDiff(a,det2x2,detNp2x2, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [52]:
countDiff(a,det3x3,detNp3x3, e=epsilon)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [53]:
epsilon = 10



In [54]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(a,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49707
Right:  49796
Collinear:  497


In [55]:
get_ipython().run_line_magic('matplotlib', 'ipympl')
plotClassification(a,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49707
Right:  49796
Collinear:  497


In [56]:
countDiff(a,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [57]:
countDiff(a,det2x2,detNp2x2, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [58]:
countDiff(a,det3x3,detNp3x3, e=epsilon)















Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [59]:
epsilon = 0



In [60]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50065
Right:  49931
Collinear:  4


In [61]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50066
Right:  49934
Collinear:  0


In [62]:
countDiff(b,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 4


In [63]:
countDiff(b,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 4


In [64]:
countDiff(b,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [65]:
epsilon = 10**0



In [66]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50065
Right:  49931
Collinear:  4


In [67]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50066
Right:  49934
Collinear:  0


In [68]:
countDiff(b,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 4


In [69]:
countDiff(b,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 4


In [70]:
countDiff(b,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [71]:
epsilon = 10**7



In [72]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50065
Right:  49931
Collinear:  4


In [73]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50066
Right:  49934
Collinear:  0


In [74]:
countDiff(b,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 4


In [75]:
countDiff(b,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 4


In [76]:
countDiff(b,det3x3,detNp3x3, e=epsilon)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [77]:
epsilon = 10**10



In [78]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50065
Right:  49929
Collinear:  6


In [79]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50065
Right:  49929
Collinear:  6


In [80]:
countDiff(b,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 2


In [81]:
countDiff(b,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 2


In [82]:
countDiff(b,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [83]:
epsilon = 10**11



In [84]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50040
Right:  49909
Collinear:  51


In [85]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  50039
Right:  49911
Collinear:  50


In [86]:
countDiff(b,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 5


In [87]:
countDiff(b,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 5


In [88]:
countDiff(b,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [89]:
epsilon = 10**12



In [90]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49827
Right:  49679
Collinear:  494


In [91]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(b,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  49826
Right:  49679
Collinear:  495


In [92]:
countDiff(b,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 5


In [93]:
countDiff(b,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 5


In [94]:
countDiff(b,det3x3,detNp3x3, e=epsilon)





























Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [95]:
epsilon = 10**-14



In [96]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  501
Right:  499
Collinear:  0


In [97]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  501
Right:  499
Collinear:  0


In [98]:
countDiff(c,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [99]:
countDiff(c,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [100]:
countDiff(c,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [101]:
epsilon = 10**-7



In [102]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  501
Right:  499
Collinear:  0


In [103]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  501
Right:  499
Collinear:  0


In [104]:
countDiff(c,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [105]:
countDiff(c,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [106]:
countDiff(c,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [107]:
epsilon = 10**-2



In [108]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  501
Right:  499
Collinear:  0


In [109]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  501
Right:  499
Collinear:  0


In [110]:
countDiff(c,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [111]:
countDiff(c,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [112]:
countDiff(c,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [113]:
epsilon = 10**0



In [114]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  499
Right:  498
Collinear:  3


In [115]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(c,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  499
Right:  498
Collinear:  3


In [116]:
countDiff(c,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [117]:
countDiff(c,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [118]:
countDiff(c,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [119]:
epsilon = 0

In [120]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det2x2,e = epsilon)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  148
Right:  146
Collinear:  706


In [121]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det3x3,e= epsilon)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  155
Right:  419
Collinear:  426


In [122]:
countDiff(d,det2x2,det3x3, e=epsilon)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 690


In [123]:
countDiff(d,det2x2,detNp3x3, e=epsilon)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 763


In [124]:
countDiff(d,det3x3,detNp3x3, e=epsilon)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 541


In [125]:
epsilon = 10**-16



In [126]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  148
Right:  145
Collinear:  707


In [127]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  155
Right:  413
Collinear:  432


In [128]:
countDiff(d,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 687


In [129]:
countDiff(d,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 757


In [130]:
countDiff(d,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 537


In [131]:
epsilon = 10**-14



In [132]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  144
Right:  141
Collinear:  715


In [133]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  0
Right:  0
Collinear:  1000


In [134]:
countDiff(d,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 285


In [135]:
countDiff(d,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 380


In [136]:
countDiff(d,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 121


In [137]:
epsilon = 10**-12



In [138]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  89
Right:  71
Collinear:  840


In [139]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  0
Right:  0
Collinear:  1000


In [140]:
countDiff(d,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 160


In [141]:
countDiff(d,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 160


In [142]:
countDiff(d,det3x3,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [143]:
epsilon = 10**-10



In [144]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det2x2,e = epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  0
Right:  0
Collinear:  1000


In [145]:
get_ipython().run_line_magic('matplotlib', 'ipympl')

plotClassification(d,det3x3,e= epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Left:  0
Right:  0
Collinear:  1000


In [146]:
countDiff(d,det2x2,det3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [147]:
countDiff(d,det2x2,detNp3x3, e=epsilon)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0


In [148]:
countDiff(d,det3x3,detNp3x3, e=epsilon)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Difference: 0
